# Characterizing DNS Censorship in Venezuela with OONI

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime as dt
from pprint import pprint
import sys
from typing import Iterable, NamedTuple, Tuple, Union
from urllib.parse import urlparse

import pandas as pd
import IPython.display as idisplay
import ipywidgets as widgets

try:
    import netanalysis
except ModuleNotFoundError:
    !{sys.executable} -m ensurepip
    !{sys.executable} -m pip install --upgrade git+https://github.com/Jigsaw-Code/net-analysis.git

import netanalysis.ooni.bucket as ob
import netanalysis.ooni.analysis.dns as od
from netanalysis.ooni.measurement import Measurement

## Fetch and save measurement data

In [4]:
import pathlib

COUNTRY = 'VE'
# Save data under /kaggle/working/ooni_data in Kaggle or ~HOME/ooni_data otherwise 
DATA_DIR = pathlib.Path('/kaggle/working')
if not DATA_DIR.is_dir():
    DATA_DIR = pathlib.Path.home()
DATA_DIR /= 'ooni_data'
local_measurements = ob.LocalMeasurements(DATA_DIR)

In [5]:
%%time
LAST_DATE = dt.date.today()
FIRST_DATE = dt.date.today() - dt.timedelta(days=14)
bytes, cost = ob.sync_measurements(local_measurements, ob.Bucket().list_files(FIRST_DATE, LAST_DATE, None, COUNTRY), cost_usd_limit=1.00)
print(f'Download size: {bytes/2**30:0.6f} GiB\nEstimated Cost: ${cost:02f}')

ectivity/2021051905_VE_webconnectivity.n0.24.jsonl.gz [1,868,027 bytes]
Downloaded 20210519/04/VE/webconnectivity/2021051904_VE_webconnectivity.n0.0.jsonl.gz [3,764,474 bytes]
Downloaded 20210519/06/VE/tor/2021051906_VE_tor.n0.0.jsonl.gz [6,990 bytes]
Downloaded 20210519/03/VE/webconnectivity/2021051903_VE_webconnectivity.n0.7.jsonl.gz [3,388,185 bytes]
Downloaded 20210519/04/VE/webconnectivity/2021051904_VE_webconnectivity.n0.8.jsonl.gz [3,546,944 bytes]
Downloaded 20210519/05/VE/webconnectivity/2021051905_VE_webconnectivity.n0.2.jsonl.gz [3,367,102 bytes]
Downloaded 20210519/04/VE/webconnectivity/2021051904_VE_webconnectivity.n0.14.jsonl.gz [3,658,442 bytes]
Downloaded 20210519/03/VE/webconnectivity/2021051903_VE_webconnectivity.n0.8.jsonl.gz [3,502,184 bytes]
Downloaded 20210519/05/VE/webconnectivity/2021051905_VE_webconnectivity.n0.1.jsonl.gz [3,454,343 bytes]
Downloaded 20210519/04/VE/webconnectivity/2021051904_VE_webconnectivity.n0.13.jsonl.gz [3,575,881 bytes]
Downloaded 2021051

NameError: name 'total_time' is not defined

In [7]:
print(f'Download size: {bytes/2**30:0.6f} GiB\nEstimated Cost: ${cost:02f}')

Download size: 7.511285 GiB
Estimated Cost: $0.676016


In [8]:
!(cd $DATA_DIR; du -d 2 -h)

 28K	./US/sniblocking
 36K	./US/example
660K	./US/facebookmessenger
1.4M	./US/httpinvalidrequestline
 75M	./US/webconnectivity
136K	./US/meekfrontedrequeststest
764K	./US/psiphon
6.3M	./US/tor
1.4M	./US/telegram
728K	./US/dash
872K	./US/tcpconnect
5.2M	./US/ndt
636K	./US/httpheaderfieldmanipulation
2.2M	./US/whatsapp
128K	./US/vanillator
 96M	./US
244K	./BY/facebookmessenger
356K	./BY/httpinvalidrequestline
 18M	./BY/webconnectivity
228K	./BY/psiphon
280K	./BY/tor
292K	./BY/telegram
208K	./BY/dash
1.3M	./BY/ndt
184K	./BY/httpheaderfieldmanipulation
440K	./BY/whatsapp
 22M	./BY
208K	./IR/facebookmessenger
180K	./IR/httpinvalidrequestline
4.1M	./IR/webconnectivity
 12K	./IR/meekfrontedrequeststest
200K	./IR/psiphon
532K	./IR/tor
216K	./IR/telegram
156K	./IR/dash
 44K	./IR/tcpconnect
612K	./IR/urlgetter
1016K	./IR/ndt
180K	./IR/httpheaderfieldmanipulation
456K	./IR/whatsapp
 12K	./IR/vanillator
4.0K	./IR/riseupvpn
7.9M	./IR
472K	./VE/facebookmessenger
528K	./VE/httpinvalidrequestline
448M

# DNS Analysis

In [41]:
%%time
import ipaddress
def is_ip(hostname):
    try:
        ipaddress.ip_address(domain)
        return True
    except ValueError:
        return False

domains = od.DomainRepository()
evaluator: od.Evaluator = od.Evaluator(domains)
rows = []
for measurement in local_measurements.get_measurements(COUNTRY, 'webconnectivity'):
    m = Measurement(measurement)
    if m.time.date() < FIRST_DATE or m.time.date() > LAST_DATE:
        continue
    domain = m.hostname
    if is_ip(domain):
        continue
    evaluator.add_control(m)
    try:
        obs = od.get_observations(domains, m)
    except:
        print(f'Failed on measurement for domain {domain}: {m.explorer_url}')
        raise        
    rows.extend(o._asdict() for o in obs)

CPU times: user 2min 54s, sys: 4.49 s, total: 2min 59s
Wall time: 3min 3s


In [42]:
# domains_to_lookup = set()
for row in rows:
    row['resolver_ip_str'] = str(row.get('resolver_ip'))
    evaluation = evaluator.evaluate(row['domain'], row['status'], row['answers'])
    row['eval'] = evaluation
    row['has_interference'] = 1 if evaluation.startswith('BAD') else 0
    # if evaluation == 'INCONCLUSIVE_CHECK_IPS':
    #     domains_to_lookup.add(row['domain'])

# for domain_name in domains_to_lookup:
#     try:
#         path = od.resolve(domains, domain_name)
#         print(f'{domain_name}: {path}')
#         domains.get(domain_name).add_path(path)
#     except Exception as e:
#         print(f'{domain_name}: {e}')
#         pass

# for row in rows:
#     if evaluation != 'INCONCLUSIVE_CHECK_IPS':
#         continue
#     evaluation = evaluator.evaluate(row['domain'], row['status'], row['answers'])
#     if evaluation != row['eval']:
#         row['eval'] = evaluation
#         row['has_interference'] = 1 if evaluation.startswith('BAD') else 0
#         print(f'Updated {row}')

obs = pd.DataFrame.from_records(rows).sort_values(by='time', ascending=False)
del(rows)
obs

,time,client_country,client_asn,resolver_ip,resolver_asn,domain,query_type,failure,status,answers,explorer_url,resolver_ip_str,eval,has_interference
70162,2021-05-19 17:01:43+00:00,VE,8048,201.249.172.70,8048,docs.google.com,AAAA,None,OK,"([], [2607:f8b0:4008:807::200e])",https://explorer.ooni.org/measurement/20210519...,201.249.172.70,INCONCLUSIVE_CHECK_IPS,0
70161,2021-05-19 17:01:43+00:00,VE,8048,201.249.172.70,8048,docs.google.com,A,None,OK,"([], [172.217.1.110])",https://explorer.ooni.org/measurement/20210519...,201.249.172.70,INCONCLUSIVE_CHECK_IPS,0
70292,2021-05-19 17:01:28+00:00,VE,8048,201.249.172.70,8048,docs.google.com,A,None,OK,"([], [172.217.3.142])",https://explorer.ooni.org/measurement/20210519...,201.249.172.70,INCONCLUSIVE_CHECK_IPS,0
70293,2021-05-19 17:01:28+00:00,VE,8048,201.249.172.70,8048,docs.google.com,AAAA,None,OK,"([], [2607:f8b0:4008:80d::200e])",https://explorer.ooni.org/measurement/20210519...,201.249.172.70,INCONCLUSIVE_CHECK_IPS,0
70158,2021-05-19 17:01:13+00:00,VE,8048,201.249.172.70,8048,drive.google.com,AAAA,None,OK,"([], [2607:f8b0:4008:80d::200e])",https://explorer.ooni.org/measurement/20210519...,201.249.172.70,INCONCLUSIVE_CHECK_IPS,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19226,2021-05-05 01:00:06+00:00,VE,6306,200.35.65.54,6306,venezuelaaldia.com,AAAA,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20210505...,200.35.65.54,BAD_STATUS_NXDOMAIN,1
19225,2021-05-05 01:00:06+00:00,VE,6306,200.35.65.54,6306,venezuelaaldia.com,A,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20210505...,200.35.65.54,BAD_STATUS_NXDOMAIN,1
19235,2021-05-05 01:00:06+00:00,VE,21826,74.125.185.71,15169,www.cinco8.com,A,None,OK,"([], [104.196.252.123])",https://explorer.ooni.org/measurement/20210505...,74.125.185.71,OK_MATCHES_CONTROL,0
19229,2021-05-05 01:00:05+00:00,VE,8048,201.249.172.72,8048,venezuelaaldia.com,A,None,OK,"([], [172.67.156.67, 104.21.7.197])",https://explorer.ooni.org/measurement/20210505...,201.249.172.72,OK_MATCHES_CONTROL,0


In [71]:
with pd.option_context('display.max_colwidth', 300, 'display.max_rows', 300):
  display(obs[(obs['has_interference'] == 1) & (obs['eval'] != 'BAD_STATUS_generic_timeout_error')].groupby(by=['client_asn', 'resolver_asn', 'eval']).sum()) #.sort_values(by='has_interference', ascending=False)

has_interference
client_asn resolver_asn eval                                                                                      
6306       6306         BAD_STATUS_NXDOMAIN                                                                   3804
8048       8048         BAD_STATUS_NXDOMAIN                                                                    176
                        BAD_STATUS_SERVFAIL                                                                   9394
                        BAD_STATUS_connection_refused                                                           22
                        BAD_STATUS_unknown_failure: lookup [DOMAIN] on [scrubbed]: lame referral                 8
11562      15169        BAD_NON_GLOBAL_IP                                                                       75
                        BAD_STATUS_NXDOMAIN                                                                   1336
                        BAD_STATUS_SERVFAIL                                                                    560
21826      15169        BAD_NON_GLOBAL_IP                                                                     3156
                        BAD_STATUS_NXDOMAIN                                                                     44
                        BAD_STATUS_SERVFAIL                                                                      2
22313      22313        BAD_STATUS_SERVFAIL                                                                   3854
263771     15169        BAD_STATUS_NXDOMAIN                                                                     42
                        BAD_STATUS_SERVFAIL                                                                      8
           20473        BAD_STATUS_NXDOMAIN                                                                     88
                        BAD_STATUS_SERVFAIL                                                                     12
264731     0            BAD_STATUS_NXDOMAIN                                                                    768
                        BAD_STATUS_SERVFAIL                                                                     56
           27717        BAD_NON_GLOBAL_IP                                                                        4
                        BAD_STATUS_NXDOMAIN                                                                   1474
                        BAD_STATUS_SERVFAIL                                                                     66

In [64]:
with pd.option_context('display.max_colwidth', 300, 'display.max_rows', 300):
  display(obs[(obs['has_interference'] == 1) & (obs['eval'] != 'BAD_STATUS_generic_timeout_error')].groupby(by=['client_asn', 'resolver_asn', 'resolver_ip_str', 'eval']).sum()) #.sort_values(by='has_interference', ascending=False)

has_interference
client_asn resolver_asn resolver_ip_str   eval                                                                                      
6306       6306         200.35.65.54      BAD_STATUS_NXDOMAIN                                                                   3804
8048       8048         201.249.172.70    BAD_STATUS_NXDOMAIN                                                                     18
                                          BAD_STATUS_SERVFAIL                                                                    944
                                          BAD_STATUS_unknown_failure: lookup [DOMAIN] on [scrubbed]: lame referral                 2
                        201.249.172.71    BAD_STATUS_NXDOMAIN                                                                      8
                                          BAD_STATUS_SERVFAIL                                                                    680
                        201.249.172.72    BAD_STATUS_NXDOMAIN                                                                     26
                                          BAD_STATUS_SERVFAIL                                                                   1322
                        201.249.172.74    BAD_STATUS_NXDOMAIN                                                                     28
                                          BAD_STATUS_SERVFAIL                                                                   1466
                        201.249.172.75    BAD_STATUS_NXDOMAIN                                                                     22
                                          BAD_STATUS_SERVFAIL                                                                   1168
                        201.249.172.76    BAD_STATUS_NXDOMAIN                                                                     26
                                          BAD_STATUS_SERVFAIL                                                                   1356
                                          BAD_STATUS_connection_refused                                                            2
                                          BAD_STATUS_unknown_failure: lookup [DOMAIN] on [scrubbed]: lame referral                 2
                        201.249.172.77    BAD_STATUS_NXDOMAIN                                                                     18
                                          BAD_STATUS_SERVFAIL                                                                   1026
                                          BAD_STATUS_connection_refused                                                           20
                                          BAD_STATUS_unknown_failure: lookup [DOMAIN] on [scrubbed]: lame referral                 2
                        201.249.172.78    BAD_STATUS_NXDOMAIN                                                                     30
                                          BAD_STATUS_SERVFAIL                                                                   1432
                                          BAD_STATUS_unknown_failure: lookup [DOMAIN] on [scrubbed]: lame referral                 2
11562      15169        172.253.216.10    BAD_NON_GLOBAL_IP                                                                        5
                                          BAD_STATUS_NXDOMAIN                                                                     80
                                          BAD_STATUS_SERVFAIL                                                                     36
                        172.253.216.15    BAD_STATUS_NXDOMAIN                                                                      6
                        172.253.216.4     BAD_NON_GLOBAL_IP                                                                        5
                                          BAD_STATUS_NXDOMAIN                                                                 

In [70]:
with pd.option_context('display.max_colwidth', 300, 'display.max_rows', 300):
  display(obs[(obs['resolver_asn'] == 15169) & (obs['has_interference'] == 1) & (obs['eval'] != 'BAD_STATUS_generic_timeout_error')].groupby(by=['client_asn', 'resolver_asn', 'domain', 'eval']).sum().head(300)) #.sort_values(by='has_interference', ascending=False)

has_interference
client_asn resolver_asn domain                          eval                                 
11562      15169        aguacateverde1.blogspot.com     BAD_STATUS_SERVFAIL                30
                        albertonews.com                 BAD_STATUS_NXDOMAIN                30
                        alekboyd.blogspot.co.uk         BAD_STATUS_SERVFAIL                32
                        alekboyd.blogspot.com           BAD_STATUS_SERVFAIL                30
                        alnavio.com                     BAD_STATUS_NXDOMAIN                30
                        analisis24.com                  BAD_NON_GLOBAL_IP                  15
                        anonymouse.org                  BAD_STATUS_SERVFAIL                30
                        caracashosting.com              BAD_STATUS_SERVFAIL                 2
                        diariolaregion.net              BAD_STATUS_NXDOMAIN                 2
                        dolar-permuta.com               BAD_STATUS_SERVFAIL                28
                        dolarparalelo.biz               BAD_STATUS_SERVFAIL                30
                        dolarparalelo.net               BAD_STATUS_SERVFAIL                30
                        dolarparalelo.org               BAD_NON_GLOBAL_IP                  15
                        dolarparalelovenezuela.com      BAD_STATUS_SERVFAIL                30
                        dolartoday.com                  BAD_STATUS_NXDOMAIN                30
                        dolartoday.info                 BAD_STATUS_SERVFAIL                30
                        dolartoday.org                  BAD_STATUS_SERVFAIL                30
                        dollar.nu                       BAD_NON_GLOBAL_IP                  15
                        dollarparalelovenezuela.com     BAD_STATUS_SERVFAIL                30
                        eldolarparalelo.info            BAD_STATUS_SERVFAIL                30
                        elpitazo.com                    BAD_STATUS_NXDOMAIN                30
                        elpitazo.info                   BAD_STATUS_NXDOMAIN                30
                        elpitazo.net                    BAD_STATUS_NXDOMAIN                32
                        infodio.com                     BAD_STATUS_SERVFAIL                32
                        liberal-venezolano.blogspot.com BAD_STATUS_SERVFAIL                30
                        livestream.com                  BAD_STATUS_NXDOMAIN                32
                        maduradas.com                   BAD_STATUS_NXDOMAIN                 2
                        miconvive.com                   BAD_STATUS_NXDOMAIN                30
                        minuto30.com                    BAD_STATUS_NXDOMAIN                30
                        muerachavez.blogspot.com        BAD_NON_GLOBAL_IP                  15
                        noticiaaldia.com                BAD_STATUS_NXDOMAIN                30
                        noticiasvenezuela.org           BAD_STATUS_NXDOMAIN                32
                        ovario2.com                     BAD_STATUS_NXDOMAIN                30
                        presidenciave.com               BAD_STATUS_NXDOMAIN                32
                        primicia.com.ve                 BAD_STATUS_NXDOMAIN                 2
                        psiphon.ca                      BAD_STATUS_NXDOMAIN                62
                        puntodecorte.com                BAD_STATUS_NXDOMAIN                32
                        pvenezuela.com                  BAD_STATUS_NXDOMAIN                32
                        quelacreo.com                   BAD_NON_GLOBAL_IP                  15
                        runrun.es                       BAD_STATUS_NXDOMAIN                30
                        soundcloud.com                  BAD_STATUS_NXDOMAIN                32
                        sumarium.co

In [50]:
with pd.option_context('display.max_colwidth', 300, 'display.max_rows', 100):
  display(obs[(obs['client_asn'] == 8048) & (obs['resolver_ip_str'] == '201.249.172.72') & (obs['eval'] == 'BAD_STATUS_NXDOMAIN')])

,time,client_country,client_asn,resolver_ip,resolver_asn,domain,query_type,failure,status,answers,explorer_url,resolver_ip_str,eval,has_interference
287041,2021-05-18 08:25:29+00:00,VE,8048,201.249.172.72,8048,quelacreo.com,A,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20210518T105159Z_webconnectivity_VE_8048_n1_RzI0tWKaACk47los?input=http%3A%2F%2Fquelacreo.com%2F,201.249.172.72,BAD_STATUS_NXDOMAIN,1
287042,2021-05-18 08:25:29+00:00,VE,8048,201.249.172.72,8048,quelacreo.com,AAAA,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20210518T105159Z_webconnectivity_VE_8048_n1_RzI0tWKaACk47los?input=http%3A%2F%2Fquelacreo.com%2F,201.249.172.72,BAD_STATUS_NXDOMAIN,1
302806,2021-05-18 07:19:34+00:00,VE,8048,201.249.172.72,8048,www.change.org,A,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20210518T105159Z_webconnectivity_VE_8048_n1_RzI0tWKaACk47los?input=https%3A%2F%2Fwww.change.org%2F,201.249.172.72,BAD_STATUS_NXDOMAIN,1
302807,2021-05-18 07:19:34+00:00,VE,8048,201.249.172.72,8048,www.change.org,AAAA,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20210518T105159Z_webconnectivity_VE_8048_n1_RzI0tWKaACk47los?input=https%3A%2F%2Fwww.change.org%2F,201.249.172.72,BAD_STATUS_NXDOMAIN,1
321657,2021-05-18 05:48:49+00:00,VE,8048,201.249.172.72,8048,quelacreo.com,A,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20210518T082727Z_webconnectivity_VE_8048_n1_NZefNkTvVRX2q4Fq?input=http%3A%2F%2Fquelacreo.com%2F,201.249.172.72,BAD_STATUS_NXDOMAIN,1
321658,2021-05-18 05:48:49+00:00,VE,8048,201.249.172.72,8048,quelacreo.com,AAAA,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20210518T082727Z_webconnectivity_VE_8048_n1_NZefNkTvVRX2q4Fq?input=http%3A%2F%2Fquelacreo.com%2F,201.249.172.72,BAD_STATUS_NXDOMAIN,1
254828,2021-05-16 06:27:18+00:00,VE,8048,201.249.172.72,8048,quelacreo.com,AAAA,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20210516T030018Z_webconnectivity_VE_8048_n1_ezd3psJZHs95xqVv?input=http%3A%2F%2Fquelacreo.com%2F,201.249.172.72,BAD_STATUS_NXDOMAIN,1
254827,2021-05-16 06:27:18+00:00,VE,8048,201.249.172.72,8048,quelacreo.com,A,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20210516T030018Z_webconnectivity_VE_8048_n1_ezd3psJZHs95xqVv?input=http%3A%2F%2Fquelacreo.com%2F,201.249.172.72,BAD_STATUS_NXDOMAIN,1
591900,2021-05-14 08:39:07+00:00,VE,8048,201.249.172.72,8048,quelacreo.com,A,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20210514T104921Z_webconnectivity_VE_8048_n1_VdNEUO5EB8XAnooQ?input=http%3A%2F%2Fquelacreo.com%2F,201.249.172.72,BAD_STATUS_NXDOMAIN,1
591901,2021-05-14 08:39:07+00:00,VE,8048,201.249.172.72,8048,quelacreo.com,AAAA,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20210514T104921Z_webconnectivity_VE_8048_n1_VdNEUO5EB8XAnooQ?input=http%3A%2F%2Fquelacreo.com%2F,201.249.172.72,BAD_STATUS_NXDOMAIN,1


In [51]:
[e for e in evaluator._errors if e[0] == 'quelacreo.com']

[('quelacreo.com', 'OK')]

In [58]:
with pd.option_context('display.max_colwidth', 300, 'display.max_rows', 150):
    display(obs[(obs['domain'] == 'quelacreo.com') & (obs['client_asn'] == 8048)])

,time,client_country,client_asn,resolver_ip,resolver_asn,domain,query_type,failure,status,answers,explorer_url,resolver_ip_str,eval,has_interference
72185,2021-05-19 09:20:46+00:00,VE,8048,201.249.172.76,8048,quelacreo.com,A,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20210519T051939Z_webconnectivity_VE_8048_n1_R8VVytp9jQ1oKtyl?input=http%3A%2F%2Fquelacreo.com%2F,201.249.172.76,BAD_STATUS_NXDOMAIN,1
72186,2021-05-19 09:20:46+00:00,VE,8048,201.249.172.76,8048,quelacreo.com,AAAA,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20210519T051939Z_webconnectivity_VE_8048_n1_R8VVytp9jQ1oKtyl?input=http%3A%2F%2Fquelacreo.com%2F,201.249.172.76,BAD_STATUS_NXDOMAIN,1
71686,2021-05-19 08:29:46+00:00,VE,8048,201.249.172.74,8048,quelacreo.com,A,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20210519T062327Z_webconnectivity_VE_8048_n1_lEshG04TAbA0Do3j?input=http%3A%2F%2Fquelacreo.com%2F,201.249.172.74,BAD_STATUS_NXDOMAIN,1
71687,2021-05-19 08:29:46+00:00,VE,8048,201.249.172.74,8048,quelacreo.com,AAAA,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20210519T062327Z_webconnectivity_VE_8048_n1_lEshG04TAbA0Do3j?input=http%3A%2F%2Fquelacreo.com%2F,201.249.172.74,BAD_STATUS_NXDOMAIN,1
64662,2021-05-19 08:11:10+00:00,VE,8048,201.249.172.71,8048,quelacreo.com,AAAA,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20210519T111518Z_webconnectivity_VE_8048_n1_R30DugQkUzQuhIth?input=http%3A%2F%2Fquelacreo.com%2F,201.249.172.71,BAD_STATUS_NXDOMAIN,1
64661,2021-05-19 08:11:10+00:00,VE,8048,201.249.172.71,8048,quelacreo.com,A,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20210519T111518Z_webconnectivity_VE_8048_n1_R30DugQkUzQuhIth?input=http%3A%2F%2Fquelacreo.com%2F,201.249.172.71,BAD_STATUS_NXDOMAIN,1
47759,2021-05-19 06:58:55+00:00,VE,8048,201.249.172.76,8048,quelacreo.com,AAAA,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20210519T030025Z_webconnectivity_VE_8048_n1_ujkbTyRkD3PWrT8y?input=http%3A%2F%2Fquelacreo.com%2F,201.249.172.76,BAD_STATUS_NXDOMAIN,1
47758,2021-05-19 06:58:55+00:00,VE,8048,201.249.172.76,8048,quelacreo.com,A,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20210519T030025Z_webconnectivity_VE_8048_n1_ujkbTyRkD3PWrT8y?input=http%3A%2F%2Fquelacreo.com%2F,201.249.172.76,BAD_STATUS_NXDOMAIN,1
53643,2021-05-19 05:54:51+00:00,VE,8048,201.249.172.75,8048,quelacreo.com,AAAA,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20210519T090056Z_webconnectivity_VE_8048_n1_0AXxrwnoISedL4TX?input=http%3A%2F%2Fquelacreo.com%2F,201.249.172.75,BAD_STATUS_NXDOMAIN,1
53642,2021-05-19 05:54:51+00:00,VE,8048,201.249.172.75,8048,quelacreo.com,A,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20210519T090056Z_webconnectivity_VE_8048_n1_0AXxrwnoISedL4TX?input=http%3A%2F%2Fquelacreo.com%2F,201.249.172.75,BAD_STATUS_NXDOMAIN,1


In [48]:
top_domains = obs.groupby(by='domain')['has_interference'].sum().sort_values(ascending=False)
top_ases = obs.value_counts(subset='client_asn')
top_statuses = obs.value_counts(subset='status')
top_evals = obs.value_counts(subset='eval')

display_columns = []
for d in [top_domains[:30], top_ases, top_statuses, top_evals]:
    w = widgets.Output(layout=widgets.Layout(margin='10px'))
    w.append_display_data(d)
    display_columns.append(w)
widgets.HBox(display_columns, layout=widgets.Layout(background='black'))

In [10]:
pivot = pd.pivot_table(obs, index=['domain'], columns=['client_asn', 'eval'], values=[], aggfunc=len, fill_value=0)
# pivot = pivot.reindex(top_sites.index).sort_index(axis='columns', level=1, ascending=False, key=lambda l: top_statuses[l])[top_ases.index]
pivot = pivot.reindex(top_domains.index).reindex(pd.MultiIndex.from_product([top_ases.index, top_evals.index]), axis='columns', fill_value=0)
with pd.option_context('display.max_rows', 150, 'display.max_columns', 24):
    display(pivot.iloc[:100, :24])

client_asn                                  21826                         \
eval                           OK_MATCHES_CONTROL INCONCLUSIVE_CHECK_IPS   
domain                                                                     
moqavemat.ir                                    0                      0   
psiphon.ca                                     12                     12   
www.coinbase.com                               12                     12   
occupystreams.org                              13                      0   
www.tunnelbear.com                             12                      0   
www.wikispaces.com                              0                     10   
www.teenhealthfx.com                           26                      0   
www.tripod.lycos.com                           26                      0   
pastebin.com                                   26                     26   
anonymouse.org                                 25                      0   
www.weebly.com                                 26                      0   
www.reddit.com                                 26                      0   
archive.is                                      0                     22   
puntodecorte.com                                0                      0   
www.efindlove.com                              25                      0   
www.helvetas.ch                                26                      0   
sunoticiero.com                                 0                      0   
www.angelfire.lycos.com                        26                      0   
www.icftu.org                                  24                      0   
www.topcities.com                              26                      0   
www.siecus.org                                 26                      0   
www.familiesaretalking.org                     26                      0   
www.exmormon.org                               26                      0   
6topoder.com                                    0                      0   
www.blogsome.com                               25                      0   
www.socom.mil                                  21                      0   
www.people.com.cn                               0                     26   
www.pridemedia.com                             26                      0   
www.ifj.org                                    25                      0   
www.maduradas.com                               0                      0   
albertonews.com                                 0                      0   
dolarparalelovenezuela.com                      0                      0   
www.ilhr.org                                   25                      0   
dolartoday.info                                 0                      0   
www.mombian.com                                24                      0   
www.phenoelit.org                              24                      0   
alnavio.com                                     1                      1   
kids.yahoo.com                                  0                     25   
www.ntn24.com                                   0                      0   
mainnet.infura.io                              26                      0   
www.partypoker.net                             26                      0   
mastodon.host                                  25                      0   
www.netaddress.com                             53                      0   
minuto30.com                                    0                      0   
www.caraota.net                                 0                      0   
www.peta.org                                   25                      0   
www.mywebcalls.com                             25                      0   
myabortionstory.tumblr.com                     26                      0   
instinctmagazine.com                           25                      0   
www.moderndrunkardmagazine.com                 2

In [12]:
with pd.option_context('display.max_colwidth', 300, 'display.max_rows', 100):
    display(obs[obs['domain'] == 'www.tunnelbear.com'].drop(columns=['domain']).sort_values(by=['client_asn', 'resolver_asn', 'resolver_ip', 'time']))

,time,client_country,client_asn,resolver_ip,resolver_asn,query_type,failure,status,answers,explorer_url,eval,has_interference
92815,2020-11-23 06:41:08+00:00,VE,6306,186.24.222.204,6306,A,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20201123T050108Z_webconnectivity_VE_6306_n1_tgVTgtl111Pyhuz5?input=https%3A%2F%2Fwww.tunnelbear.com%2F,BAD_STATUS_NXDOMAIN,1
92816,2020-11-23 06:41:08+00:00,VE,6306,186.24.222.204,6306,AAAA,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20201123T050108Z_webconnectivity_VE_6306_n1_tgVTgtl111Pyhuz5?input=https%3A%2F%2Fwww.tunnelbear.com%2F,BAD_STATUS_NXDOMAIN,1
48063,2020-11-20 06:33:11+00:00,VE,6306,186.166.131.54,6306,AAAA,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20201120T050056Z_webconnectivity_VE_6306_n1_SumX5KILeq1HhyUw?input=https%3A%2F%2Fwww.tunnelbear.com%2F,BAD_STATUS_NXDOMAIN,1
48062,2020-11-20 06:33:11+00:00,VE,6306,186.166.131.54,6306,A,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20201120T050056Z_webconnectivity_VE_6306_n1_SumX5KILeq1HhyUw?input=https%3A%2F%2Fwww.tunnelbear.com%2F,BAD_STATUS_NXDOMAIN,1
99492,2020-11-24 06:34:28+00:00,VE,6306,186.166.131.54,6306,A,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20201124T050507Z_webconnectivity_VE_6306_n1_WELrcTk7RyiN6LAj?input=https%3A%2F%2Fwww.tunnelbear.com%2F,BAD_STATUS_NXDOMAIN,1
99493,2020-11-24 06:34:28+00:00,VE,6306,186.166.131.54,6306,AAAA,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20201124T050507Z_webconnectivity_VE_6306_n1_WELrcTk7RyiN6LAj?input=https%3A%2F%2Fwww.tunnelbear.com%2F,BAD_STATUS_NXDOMAIN,1
8226,2020-11-21 05:48:48+00:00,VE,6306,186.166.131.55,6306,A,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20201121T050305Z_webconnectivity_VE_6306_n1_H8jLO6sjqROtdezE?input=https%3A%2F%2Fwww.tunnelbear.com%2F,BAD_STATUS_NXDOMAIN,1
8227,2020-11-21 05:48:48+00:00,VE,6306,186.166.131.55,6306,AAAA,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20201121T050305Z_webconnectivity_VE_6306_n1_H8jLO6sjqROtdezE?input=https%3A%2F%2Fwww.tunnelbear.com%2F,BAD_STATUS_NXDOMAIN,1
16723,2020-11-19 05:24:28+00:00,VE,6306,186.166.131.56,6306,A,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20201119T050305Z_webconnectivity_VE_6306_n1_mPnQ2fz5N6JMQ6Mo?input=https%3A%2F%2Fwww.tunnelbear.com%2F,BAD_STATUS_NXDOMAIN,1
16724,2020-11-19 05:24:28+00:00,VE,6306,186.166.131.56,6306,AAAA,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20201119T050305Z_webconnectivity_VE_6306_n1_mPnQ2fz5N6JMQ6Mo?input=https%3A%2F%2Fwww.tunnelbear.com%2F,BAD_STATUS_NXDOMAIN,1


In [59]:
!dig @200.35.192.12 torproject.org


; <<>> DiG 9.10.6 <<>> @200.35.192.12 torproject.org
; (1 server found)
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 50158
;; flags: qr rd; QUERY: 1, ANSWER: 0, AUTHORITY: 5, ADDITIONAL: 1
;; WARNING: recursion requested but not available

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4096
;; QUESTION SECTION:
;torproject.org.			IN	A

;; AUTHORITY SECTION:
torproject.org.		43963	IN	NS	nsp.dnsnode.net.
torproject.org.		43963	IN	NS	ns1.torproject.org.
torproject.org.		43963	IN	NS	ns3.torproject.org.
torproject.org.		43963	IN	NS	ns4.torproject.org.
torproject.org.		43963	IN	NS	ns5.torproject.org.

;; Query time: 63 msec
;; SERVER: 200.35.192.12#53(200.35.192.12)
;; WHEN: Fri Nov 20 12:34:30 EST 2020
;; MSG SIZE  rcvd: 144



In [24]:
obs[['client_asn', 'resolver_asn', 'resolver_ip']].value_counts()

client_asn  resolver_asn  resolver_ip    
22313       22313         200.35.192.12      3878
21826       15169         172.253.242.44     3827
8048        8048          201.249.172.71     3641
                          201.249.172.74     1991
                          201.249.172.72     1984
                          201.249.172.77     1962
                          201.249.172.70     1942
6306        6306          186.166.131.56     1942
8048        8048          201.249.215.7      1925
                          201.249.215.6      1922
21826       15169         172.253.242.41     1921
                          74.125.184.10      1919
                          172.253.7.65       1917
                          74.125.75.72       1915
                          74.125.77.75       1914
                          172.253.7.68       1913
                          172.253.7.13       1913
                          74.125.78.130      1913
                          172.253.242.105    1912
        

In [102]:
obs[['client_asn', 'resolver_asn', 'eval']].value_counts()

client_asn  resolver_asn  eval                  
12880       0             OK_MATCHES_CONTROL        2640
50810       0             OK_MATCHES_CONTROL        2497
197207      20473         OK_MATCHES_CONTROL        1183
50810       0             INCONCLUSIVE_CHECK_IPS     492
12880       0             INCONCLUSIVE_CHECK_IPS     415
                                                    ... 
50810       58224         BAD_NON_GLOBAL_IP            1
197207      12440         BAD_NON_GLOBAL_IP            1
49100       49100         BAD_NON_GLOBAL_IP            1
197207      42            INCONCLUSIVE_CHECK_IPS       1
56402       56402         BAD_NON_GLOBAL_IP            1
Length: 111, dtype: int64

In [352]:
!dig @201.249.172.77 eldolarparalelo.info


; <<>> DiG 9.10.6 <<>> @201.249.172.77 eldolarparalelo.info
; (1 server found)
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: SERVFAIL, id: 57965
;; flags: qr rd; QUERY: 1, ANSWER: 0, AUTHORITY: 0, ADDITIONAL: 1
;; WARNING: recursion requested but not available

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4096
;; QUESTION SECTION:
;eldolarparalelo.info.		IN	A

;; Query time: 277 msec
;; SERVER: 201.249.172.77#53(201.249.172.77)
;; WHEN: Sat Nov 14 02:29:30 EST 2020
;; MSG SIZE  rcvd: 49



In [353]:
!dig @201.249.172.77 example.com


; <<>> DiG 9.10.6 <<>> @201.249.172.77 example.com
; (1 server found)
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: REFUSED, id: 50572
;; flags: qr rd; QUERY: 1, ANSWER: 0, AUTHORITY: 0, ADDITIONAL: 1
;; WARNING: recursion requested but not available

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4096
;; QUESTION SECTION:
;example.com.			IN	A

;; Query time: 72 msec
;; SERVER: 201.249.172.77#53(201.249.172.77)
;; WHEN: Sat Nov 14 02:29:44 EST 2020
;; MSG SIZE  rcvd: 40



In [112]:
obs[obs['domain'] == 'getoutline.org']

,time,client_country,client_asn,resolver_ip,resolver_asn,domain,query_type,failure,status,answers,explorer_url,eval,has_interference
6422,2020-11-14 11:30:52+00:00,IR,12880,2.188.21.2,0,getoutline.org,A,None,OK,"([], [216.239.34.21, 216.239.32.21, 216.239.36...",https://explorer.ooni.org/measurement/20201114...,OK_MATCHES_CONTROL,0
6423,2020-11-14 11:30:47+00:00,IR,12880,2.188.21.2,0,getoutline.org,A,None,OK,"([], [216.239.34.21, 216.239.38.21, 216.239.36...",https://explorer.ooni.org/measurement/20201114...,OK_MATCHES_CONTROL,0
7433,2020-11-13 09:32:11+00:00,IR,44244,127.0.0.2,44244,getoutline.org,AAAA,None,OK,"([], [2001:4860:4802:32::15, 2001:4860:4802:36...",https://explorer.ooni.org/measurement/20201113...,INCONCLUSIVE_CHECK_IPS,0
7432,2020-11-13 09:32:11+00:00,IR,44244,127.0.0.2,44244,getoutline.org,A,None,OK,"([], [216.239.34.21, 216.239.36.21, 216.239.32...",https://explorer.ooni.org/measurement/20201113...,OK_MATCHES_CONTROL,0
7844,2020-11-13 02:25:20+00:00,IR,197207,127.0.0.2,20473,getoutline.org,A,None,OK,"([], [216.239.34.21, 216.239.36.21, 216.239.32...",https://explorer.ooni.org/measurement/20201113...,OK_MATCHES_CONTROL,0
4302,2020-11-09 12:16:18+00:00,IR,43754,127.0.0.2,43754,getoutline.org,AAAA,None,OK,"([], [2001:4860:4802:36::15, 2001:4860:4802:34...",https://explorer.ooni.org/measurement/20201109...,INCONCLUSIVE_CHECK_IPS,0
4301,2020-11-09 12:16:18+00:00,IR,43754,127.0.0.2,43754,getoutline.org,A,None,OK,"([], [216.239.36.21, 216.239.34.21, 216.239.38...",https://explorer.ooni.org/measurement/20201109...,OK_MATCHES_CONTROL,0
4128,2020-11-09 01:55:38+00:00,IR,50810,185.5.159.143,0,getoutline.org,A,None,OK,"([], [216.239.36.21, 216.239.32.21, 216.239.34...",https://explorer.ooni.org/measurement/20201108...,OK_MATCHES_CONTROL,0
1620,2020-11-08 22:39:20+00:00,IR,50810,172.253.215.13,0,getoutline.org,A,None,OK,"([], [216.239.38.21, 216.239.32.21, 216.239.34...",https://explorer.ooni.org/measurement/20201108...,OK_MATCHES_CONTROL,0
1563,2020-11-08 22:39:15+00:00,IR,50810,172.253.215.13,0,getoutline.org,A,None,OK,"([], [216.239.38.21, 216.239.34.21, 216.239.36...",https://explorer.ooni.org/measurement/20201108...,OK_MATCHES_CONTROL,0


In [107]:
list(obs[obs['domain'] == 'occupystreams.org'].explorer_url)

['https://explorer.ooni.org/measurement/20201115T194951Z_webconnectivity_IR_12880_n1_xqETuH8J04j66IzK?input=http%3A%2F%2Foccupystreams.org%2F',
 'https://explorer.ooni.org/measurement/20201114T193120Z_webconnectivity_IR_12880_n1_hxvro3KVFCFikwh9?input=http%3A%2F%2Foccupystreams.org%2F',
 'https://explorer.ooni.org/measurement/20201114T101626Z_webconnectivity_IR_12880_n1_BCz7kLvLynnn3x7L?input=http%3A%2F%2Foccupystreams.org%2F',
 'https://explorer.ooni.org/measurement/20201113T002153Z_webconnectivity_IR_197207_n1_DVKMi6cXW1JLoyo4?input=http%3A%2F%2Foccupystreams.org%2F',
 'https://explorer.ooni.org/measurement/20201111T195820Z_webconnectivity_IR_12880_n1_4O9jjJ2y5IBuTs4B?input=http%3A%2F%2Foccupystreams.org%2F',
 'https://explorer.ooni.org/measurement/20201108T214526Z_webconnectivity_IR_50810_n1_DJlMYn9WesLpsi6x?input=http%3A%2F%2Foccupystreams.org%2F',
 'https://explorer.ooni.org/measurement/20201108T214221Z_webconnectivity_IR_50810_n1_7eoPoM8KU2SCgFLO?input=http%3A%2F%2Foccupystreams.

In [106]:
for asn, asn_df in obs.groupby(by='client_asn'):
    print(f'======= AS{asn} ======')
    for result, result_df in asn_df.groupby(by='eval'):
        print(f'----- {result} ------')
        with pd.option_context('display.min_rows', 50):
            display(result_df['domain'].value_counts())
        print()
    print()
    

======= AS12880 ======
----- BAD_NON_GLOBAL_IP ------


twitter.com                     8
beeg.com                        5
ultrasurf.us                    4
bridges.torproject.org          4
adultfriendfinder.com           4
anon.inf.tu-dresden.de          4
freenetproject.org              4
myspace.com                     4
ooni.torproject.org             3
www.bbc.com                     3
openvpn.net                     3
blogspot.com                    3
proxy.org                       2
thepiratebay.se                 2
strongvpn.com                   2
psiphon.ca                      2
dentist.blogspot.com            2
plus.google.com                 2
metasploit.com                  2
lgbt.foundation                 2
trashy.com                      2
download.cnet.com               2
www.xroxy.com                   2
proxify.com                     2
spys.ru                         2
                               ..
www.anonymizer.com              1
dooush.blogspot.com             1
bit.ly                          1
nesvan1.wordpr


----- BAD_STATUS_NXDOMAIN ------


www.meaphone.com    1
Name: domain, dtype: int64


----- BAD_STATUS_SERVFAIL ------


...

www.darpa.mil       2
www.hra-news.org    2
Name: domain, dtype: int64


----- INCONCLUSIVE_CHECK_IPS ------


europedatingonline.com    1
www.kampain.info          1
Name: domain, dtype: int64


----- OK_MATCHES_CONTROL ------


vimeo.com                 1
europedatingonline.com    1
www.anonymizer.ru         1
www.kampain.info          1
www.iran-e-azad.org       1
Name: domain, dtype: int64

In [134]:
od.resolve(domains, 'www.match.com')

DnsPath(cnames=[Domain('us-wac.match.com')], ips=[IPv4Address('208.83.242.49')])

In [19]:
socket.gethostbyname_ex('www.match.com')

('us-wac.match.com', ['www.match.com', 'us.match.com'], ['208.83.242.49'])

In [113]:
socket.gethostbyname_ex('no.match.cosm')

gaierror: [Errno 8] nodename nor servname provided, or not known

In [114]:
socket.gethostbyname_ex('occupystreams.org')

gaierror: [Errno 8] nodename nor servname provided, or not known

In [115]:
socket.getaddrinfo('occupystreams.org', 0, proto=socket.IPPROTO_TCP, flags=socket.AI_CANONNAME)

gaierror: [Errno 8] nodename nor servname provided, or not known

In [116]:
socket.getaddrinfo('invalid.example.com', 0, proto=socket.IPPROTO_TCP, flags=socket.AI_CANONNAME)

gaierror: [Errno 8] nodename nor servname provided, or not known

In [119]:
import dns.resolver

ModuleNotFoundError: No module named 'dns.resolver'; 'dns' is not a package

In [118]:
!pip install dnspython

     |████████████████████████████████| 208 kB 3.8 MB/s 
You should consider upgrading via the '/Users/fortuna/firehook/net-analysis/.venv/bin/python -m pip install --upgrade pip' command.
